<a href="https://colab.research.google.com/github/Gabucado/2021-1/blob/master/AC09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio de Trabajo en Clases 09: Reconocimiento de Objetos en Proyecto P02

Basarse en el código anterior y sus parámetros (BATCH, EARLY_STOP, num_epochs, etc.)

In [1]:
# Funciones

from IPython.display import clear_output

import torch
import os, sys, fnmatch
import numpy as np
from torchvision import models
import matplotlib.pyplot as plt

util = 'pretrain_utils'
utilpy = util+'2.py'
if os.path.isfile(utilpy):
  del sys.modules[util]
  os.remove(utilpy)
  print(util+' removed.')
print(util+' ready to load.')

!wget https://www.dropbox.com/s/5nnub5axbj20e2m/pretrain_utils2.py
!cp pretrain_utils2.py pretrain_utils.py
from pretrain_utils import loaderdef,modeldef,showimages,evaluate,megatrain,printaccm,get_predictions,plot_confusion_matrix
print('pretrain_utils module has been loaded')

!pip install Augmentor
import Augmentor

# INCLUDE HERE THE AUGMENTATION FUNCTIONS
def augment(PATH_ORIGINAL, PATH_AUGMENTED, set_name, N):
  ldir = dirdir(PATH_ORIGINAL+set_name)
  for i in range(len(ldir)):
    print('processing '+ldir[i])
    p0 = Augmentor.Pipeline(PATH_ORIGINAL+set_name+'/'+ldir[i],output_directory='../../../'+PATH_AUGMENTED+set_name+'/'+ldir[i])
    p0.set_seed(0)
    #p0.rotate90(probability=0.25)
    #p0.rotate180(probability=0.25)
    #p0.rotate270(probability=0.25)
    p0.flip_left_right(probability=0.5)
    #p0.skew(probability=0.5)
    #p0.flip_random(probability=0.25)
    p0.sample(N)

def dirfiles(fpath,fext):
    fnames = fnmatch.filter(sorted(os.listdir(fpath)),fext)
    if '.DS_Store' in fnames:
        fnames.remove('.DS_Store')
    return fnames
def dirdir(fpath):
    fnames = sorted(os.listdir(fpath))
    if '.DS_Store' in fnames:
        fnames.remove('.DS_Store')
    return fnames

def imagesxclass(fpath,fext='*.jpg'):
  fdir = dirdir(fpath)
  for i in range(len(fdir)):
    x = dirfiles(fpath+'/'+fdir[i]+'/',fext)
    print(i,fdir[i],len(x))


clear_output()


# 1. Lectura de la base de datos del proyecto

 La base de datos está en el siguiente enlace:
 https://www.dropbox.com/s/0ezq2uv1c4wff9y/mochis.zip


In [2]:
if not os.path.exists("./mochis"):
  os.system(""" wget https://www.dropbox.com/s/0ezq2uv1c4wff9y/mochis.zip && unzip -qq mochis.zip && rm mochis.zip """)
  print("Datos descargados exitosamente")
print("Datos listos")


Datos descargados exitosamente
Datos listos


# 2. Despliegue de imagen de ejemplo y sus *bounding boxes*.

Se desplegará la imagen 

In [ ]:
format_path = lambda g, d, i: f"G{g:02}_{d}{i:03}.png"
imgpath = format_path(6, 3, 1)
img = cv2

Basarse en el código anterior y sus parámetros (BATCH, EARLY_STOP, num_epochs, etc.)

1) [1 punto] Lea la base de datos del proyecto 'mochis' disponible aquí: 

https://www.dropbox.com/s/0ezq2uv1c4wff9y/mochis.zipEnlaces a un sitio externo.

2) [1 punto] Despliegue alguna imagen con sus bounding boxes (cada tipo de objeto en un color distinto, ejemplo: mochila-rojo, cabeza-verde, persona-azul).

3) [4 puntos] Genere la Base de Datos para clasificación entre objetos masculinos/femeninos:

Escoja 500 imágenes de la base de datos 'mochis'
De estas 500 imágenes, 70% serán destinadas a training, 10% a validation y 20% a testing.
Escoja un objeto a clasificar (mochila, cabeza, o persona)
Construya la base de datos con imágenes de 64x64 pixeles del objeto seleccionado
4) [4 puntos] Genere una nueva base de datos balanceada en el training y en validation usando la librería "Augmentor" (el conjunto de testing no se cambia, debe contener sólo imágenes reales, no aumentadas)

5) [2 puntos] Usando la función "loaderdef", cargue los datos de training/validation/testing de esta nueva base de datos de tal forma que no se generen imágenes invertidas.

6) [5 puntos] Entrene al menos los siguientes tres modelos:

CNN small (cnn-s)
VGG-19
ResNet-34
7) [1 punto] Encuentre la Matriz de Confusión en el Testing del mejor modelo